In [ ]:
from ultralytics import YOLO
import datetime

# 현재 날짜/시간을 기반으로 폴더 이름 생성
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
train_name = f"train_{timestamp}_1000"  # 예: train_20250224_153045

# YOLOv8 모델 로드 (yolov8n: Nano 버전)
model = YOLO("yolov8n.pt")

# 학습 실행
model.train(
    data="../dataset/dataset.yaml",
    epochs=50,
    batch=16,
    imgsz=640,
    freeze=10,
    lr0=0.001,
    save_period=10,
    project="runs/detect",  # 프로젝트 경로 지정
    name=train_name,        # 현재 학습별 고유한 이름 지정
    exist_ok=False          # 같은 이름이 있어도 덮어쓰지 않고 새로운 폴더 생성
)

# 학습 완료 메시지 출력
print(f"학습 완료! 결과는 runs/detect/{train_name} 폴더에 저장되었습니다.")

# 학습된 모델의 best 가중치 경로
best_model_path = f"runs/detect/{train_name}/weights/best.pt"

# Valid 성능 평가
print("\n [Valid 데이터셋 평가 중...]")
valid_results = model.val(data="../dataset/dataset.yaml")  # 기본적으로 valid 데이터셋 평가
print(f"Valid 데이터셋 성능:\n{valid_results}")

# Test 데이터셋 평가
print("\n [Test 데이터셋 평가 중...]")
test_results = model.val(data="../dataset/dataset.yaml", split="test")  # test 데이터셋 평가
print(f"Test 데이터셋 성능:\n{test_results}")


In [ ]:
from ultralytics import YOLO
import datetime

# 현재 날짜/시간을 기반으로 폴더 이름 생성
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
train_name = f"train_{timestamp}_A1000"

# 기존 best.pt 모델 로드 (이전까지 학습된 가중치 유지)
model = YOLO("/home/alpaco/lyj/train_yolov8_finetune/runs/detect/train_20250225_191809_1000/weights/best.pt")

# 추가 학습 실행 (person 클래스만 추가 학습)
model.train(
    data="../dataset/dataset.yaml", 
    epochs=50,
    batch=16,
    imgsz=640,  
    lr0=0.001,
    freeze=[0,1,2,3,4,5,6,7,8],  # Backbone + Neck 동결, 기존 클래스 유지
    save_period=10,
    project="runs/detect",
    name=train_name,
    exist_ok=False
)

# 학습 완료 메시지 출력
print(f"학습 완료! 결과는 runs/detect/{train_name} 폴더에 저장되었습니다.")

# 학습된 모델의 best 가중치 경로
best_model_path = f"runs/detect/{train_name}/weights/best.pt"

# Valid 성능 평가
print("\n [Valid 데이터셋 평가 중...]")
valid_results = model.val(data="../dataset/dataset.yaml")
print(f"Valid 데이터셋 성능:\n{valid_results}")

# Test 데이터셋 평가
print("\n [Test 데이터셋 평가 중...]")
test_results = model.val(data="../dataset/dataset.yaml")  # test도 함께 평가
print(f"Test 데이터셋 성능:\n{test_results}")


In [ ]:
from ultralytics import YOLO

# IoU Threshold를 낮춰서 겹쳐진 객체도 탐지되도록 설정 (기본값 0.5 → 0.4)


# 훈련된 모델 로드
model = YOLO("/home/alpaco/lyj/train_yolov8_finetune/runs/detect/train_20250226_184337_A1000/weights/best.pt")

# 영상 파일에서 객체 탐지 수행
model.predict(
    source="/home/alpaco/lyj/video/test_yj.mp4",  # 탐지할 영상 파일
    save=True, 
    #mgsz=1280,  # 이미지 크기
    # conf=0.3,  # 신뢰도 60% 이상인 경우만 탐지 (오탐 방지) 0.25 -> 0.3
    # iou=0.3,  # IoU 임계값 (겹치는 박스 제거 기준)   0.4 -> 0.3
    device="cuda",  # GPU 사용
    half=True,  # FP16 연산 (메모리 절약 + 속도 향상)
    # vid_stride=2  # 프레임 건너뛰기 (속도 향상)
)
